# **Stacking Regressor**

## **Importing Libararies**

In [180]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import StackingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRFRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

## **Data Overview**

In [181]:
df = pd.read_csv('Dataset salary 2024.csv')
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2024,SE,FT,AI Engineer,202730,USD,202730,US,0,US,M
1,2024,SE,FT,AI Engineer,92118,USD,92118,US,0,US,M
2,2024,SE,FT,Data Engineer,130500,USD,130500,US,0,US,M
3,2024,SE,FT,Data Engineer,96000,USD,96000,US,0,US,M
4,2024,SE,FT,Machine Learning Engineer,190000,USD,190000,US,0,US,M


In [182]:
df.shape

(16534, 11)

In [183]:
df.isnull().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [184]:
df.describe(include='all')

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
count,16534.000000,16534,16534,16534,1.653400e+04,16534,16534.000000,16534,16534.000000,16534,16534
unique,NaN,4,4,155,NaN,23,NaN,88,NaN,77,3
top,NaN,SE,FT,Data Engineer,NaN,USD,NaN,US,NaN,US,M
freq,NaN,10670,16454,3464,NaN,15294,NaN,14467,NaN,14518,15306
mean,2023.226866,NaN,NaN,NaN,1.637270e+05,NaN,149686.777973,NaN,32.003750,NaN,NaN
std,0.713558,NaN,NaN,NaN,3.402057e+05,NaN,68505.293156,NaN,46.245158,NaN,NaN
min,2020.000000,NaN,NaN,NaN,1.400000e+04,NaN,15000.000000,NaN,0.000000,NaN,NaN
25%,2023.000000,NaN,NaN,NaN,1.017630e+05,NaN,101125.000000,NaN,0.000000,NaN,NaN
50%,2023.000000,NaN,NaN,NaN,1.422000e+05,NaN,141300.000000,NaN,0.000000,NaN,NaN
75%,2024.000000,NaN,NaN,NaN,1.872000e+05,NaN,185900.000000,NaN,100.000000,NaN,NaN


In [185]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16534 entries, 0 to 16533
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           16534 non-null  int64 
 1   experience_level    16534 non-null  object
 2   employment_type     16534 non-null  object
 3   job_title           16534 non-null  object
 4   salary              16534 non-null  int64 
 5   salary_currency     16534 non-null  object
 6   salary_in_usd       16534 non-null  int64 
 7   employee_residence  16534 non-null  object
 8   remote_ratio        16534 non-null  int64 
 9   company_location    16534 non-null  object
 10  company_size        16534 non-null  object
dtypes: int64(4), object(7)
memory usage: 1.4+ MB


In [186]:
df['salary'].equals(df['salary_in_usd'])

False

## **Train Test Split**

In [187]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['salary', 
                                                                    'salary_in_usd', 'salary_currency']), 
                                                    df['salary'],
                                                    test_size=0.3,
                                                    random_state=42)

In [188]:
X_train.head(1)

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size
14159,2023,MI,FT,Data Scientist,US,100,US,M


## **Feature Engineering**

In [189]:
# numerical_columns = [0, 6]
# ordinal_columns = [8]
# nominal_columns = [1, 2, 3, 4, 5, 7]

In [201]:
df['experience_level'].value_counts()

experience_level
SE    10670
MI     4038
EN     1325
EX      501
Name: count, dtype: int64

In [190]:
numerical_columns = ['work_year', 'remote_ratio']
ordinal_columns = ['experience_level', 'company_size']
nominal_columns = ['employment_type', 'job_title', 'employee_residence', 'company_location']

In [191]:
handle_numerical = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', MinMaxScaler())
])

In [192]:
handle_ordinal = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('encode', OrdinalEncoder())
])

In [193]:
handle_nominal = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('encode', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

In [194]:
preprocessing = ColumnTransformer(transformers=[
    ('numerical', handle_numerical, numerical_columns),
    ('ordinal', handle_ordinal, ordinal_columns),
    ('nominal', handle_nominal, nominal_columns)
], remainder='passthrough')

## **Modeling and Evaluation**

In [195]:
base_models = [
    ('LinearRegression', make_pipeline(preprocessing, LinearRegression())),
    ('RandomForestRegressor', make_pipeline(preprocessing, RandomForestRegressor())),
    ('DecisionTreeRegressor', make_pipeline(preprocessing, DecisionTreeRegressor())),
    ('GradientBoostingRegressor', make_pipeline(preprocessing, GradientBoostingRegressor()))
]

In [196]:
meta_model = XGBRFRegressor()

In [197]:
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5
)

In [198]:
stacking_regressor.fit(X_train, y_train)

c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [1, 2, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [1, 2, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [1, 2, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [1, 2, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [1, 

StackingRegressor(cv=5,
                  estimators=[('LinearRegression',
                               Pipeline(steps=[('columntransformer',
                                                ColumnTransformer(remainder='passthrough',
                                                                  transformers=[('numerical',
                                                                                 Pipeline(steps=[('impute',
                                                                                                  SimpleImputer()),
                                                                                                 ('scale',
                                                                                                  MinMaxScaler())]),
                                                                                 ['work_year',
                                                                                  'remote_ratio']),
                                                                                ('ordinal',
                                                                                 Pipeline(steps=[('impute',
                                                                                                  SimpleImputer(strategy='most_frequent')),
                                                                                                 ('encode',
                                                                                                  O...
                                                 importance_type=None,
                                                 interaction_constraints=None,
                                                 max_bin=None,
                                                 max_cat_threshold=None,
                                                 max_cat_to_onehot=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 max_leaves=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 multi_strategy=None,
                                                 n_estimators=None, n_jobs=None,
                                                 num_parallel_tree=None,
                                                 objective='reg:squarederror',
                                                 random_state=None,
                                                 reg_alpha=None, ...))

In [199]:
y_pred = stacking_regressor.predict(X_test)

c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [1, 2, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [1, 2, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [1, 2, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [1, 2, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [200]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R2 Score: {r2}")

Mean Absolute Error (MAE): 57122.4019004485
Mean Squared Error (MSE): 81223055133.11938
R2 Score: 0.1286662220954895
